In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pyspark
!pip install findspark
!pip install numpy

In [3]:
import os
import re

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import concat_ws

from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StringIndexer, VectorIndexer, StringIndexerModel, IndexToString

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, LogisticRegressionModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator

spark = SparkSession.builder.appName("final_prj").getOrCreate()


In [129]:
train_data = "/content/drive/MyDrive/bigdata/bigdata-project/khoa_test/src/datasets/f-train.csv"
# train_data = "/content/drive/MyDrive/bigdata/bigdata-project/khoa_test/src/datasets/youtoxic_english_1000.csv"
test_data = "/content/drive/MyDrive/bigdata/bigdata-project/khoa_test/src/datasets/f-test.csv"

#Data clone from youtube
# videos_data = "/content/drive/MyDrive/0000_UIT_Graduation/9_Big_data/0000_Final_PRJ/0_data/videos.csv"
# comments_data = "/content/drive/MyDrive/0000_UIT_Graduation/9_Big_data/0000_Final_PRJ/0_data/comments.csv"

In [130]:
test  = pd.read_csv (test_data)
train = pd.read_csv(train_data, usecols = ['comment_text','toxic'])
# train = pd.read_csv(train_data, usecols = ['Text','IsToxic'])
print(train)

                                             comment_text  toxic
0       Explanation\nWhy the edits made under my usern...      0
1       D'aww! He matches this background colour I'm s...      0
2       Hey man, I'm really not trying to edit war. It...      0
3       "\nMore\nI can't make any real suggestions on ...      0
4       You, sir, are my hero. Any chance you remember...      0
...                                                   ...    ...
159566  ":::::And for the second time of asking, when ...      0
159567  You should be ashamed of yourself \n\nThat is ...      0
159568  Spitzer \n\nUmm, theres no actual article for ...      0
159569  And it looks like it was actually you who put ...      0
159570  "\nAnd ... I really don't think you understand...      0

[159571 rows x 2 columns]


In [48]:
def preprocess_text(text):
    """
    Take a string as input and performs the following:
    1. Remove extra spaces
    2. Remove “\n”
    3. Remove Non-English characters
    4. Remove start and end white spaces
    5. Remove single characters
    6. Remove punctuations
    7. Lowercase

    :param text: string
    :return: string
    """
    # remove extra spaces
    text = re.sub("\s\s+", " ", text)
    text = text.replace("\n", " ")
    # remove Non-English characters
    text = re.sub(r'[^\x00-\x7F]+', "", text)
    text = text.strip()
    # remove single characters
    text = re.sub(r"\s+[a-zA-Z]\s+", " ", text)
    # remove punctuations
    text = re.sub(r"[^a-zA-Z0-9]+", " ", text)
    text = text.lower()

    return text

def castBoolToInt(boolVal):
  return float(boolVal == True)

In [90]:
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [92]:
#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower()
    text=text.strip()
    text=re.compile('<.*?>').sub('', text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)
    text = re.sub('\s+', ' ', text)
    text = re.sub(r'\[[0-9]*\]',' ',text)
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text)
    text = re.sub(r'\s+',' ',text)
    return text

# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)

#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()

# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

In [131]:
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))

train['clean_text'] = train['comment_text'].apply(lambda x: finalpreprocess(x))
# train['clean_text'] = train['Text'].apply(lambda x: finalpreprocess(x))
train.head()

,comment_text,toxic,clean_text
0,Explanation\nWhy the edits made under my usern...,0,explanation edits make username hardcore metal...
1,D'aww! He matches this background colour I'm s...,0,aww match background colour seemingly stuck th...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man really try edit war guy constantly rem...
3,"""\nMore\nI can't make any real suggestions on ...",0,make real suggestion improvement wonder sectio...
4,"You, sir, are my hero. Any chance you remember...",0,sir hero chance remember page


In [132]:
def convertBoolToNumber(boolVal):
  return float(boolVal == True)

In [134]:

# train['IsToxic'] = train['IsToxic'].apply(convertBoolToNumber)
train['IsToxic'] = train['toxic'].apply(convertBoolToNumber)

In [135]:

spark_train=spark.createDataFrame(train)

spark_train.printSchema()

root
 |-- comment_text: string (nullable = true)
 |-- toxic: long (nullable = true)
 |-- clean_text: string (nullable = true)
 |-- IsToxic: double (nullable = true)



In [136]:
commentIndexer = StringIndexer(inputCol="clean_text", outputCol="comment_text_id")
spark_train=commentIndexer.fit(spark_train).transform(spark_train)

# labelsIndexer = StringIndexer(inputCol="IsToxic", outputCol="is_toxic_id")
# spark_train=labelsIndexer.fit(spark_train).transform(spark_train)



In [137]:
assembler = VectorAssembler(inputCols=['comment_text_id'], outputCol="features")
spark_train = assembler.transform(spark_train)

In [139]:
(trainingData, testData) = spark_train.randomSplit([0.7, 0.3])
print("Training Dataset Count: " + str(trainingData.count()))
# print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 111849


In [140]:
lr = LogisticRegression(featuresCol='features', labelCol='IsToxic')
lrModel = lr.fit(trainingData)

In [154]:
!pip install joblib

In [159]:
# loading dependency
# import pickle

file_path = "/content/drive/MyDrive/bigdata/bigdata-project/khoa_test/src/trained_lrmodel"
lrModel.save(file_path)

# with open('/content/drive/MyDrive/bigdata/bigdata-project/khoa_test/src/model_pkl', 'wb') as files:
#     pickle.dump(lrModel, files)

TypeError: ignored

In [160]:
predictions = lrModel.transform(testData)

In [ ]:
predictions.select("comment_text","toxic", "comment_text_id", "IsToxic", "prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 100, truncate = False)

# predictions.select("Text","IsToxic", "comment_text_id", "prediction") \
#     .orderBy("probability", ascending=False) \
#     .show(n = 1000, truncate = False)

In [161]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="IsToxic")
evaluator.evaluate(predictions)

0.8596741575945753

In [166]:
loadedLRModel = LogisticRegressionModel.load(file_path)

In [167]:
loadedLRModelPredictions = loadedLRModel.transform(testData)

In [168]:
# from pyspark.ml.evaluation import MulticlassClassificationEvaluator
loadedLRModelPredictionsEvaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="IsToxic")
loadedLRModelPredictionsEvaluator.evaluate(loadedLRModelPredictions)

0.8596741575945753

# New Section